# モデルの解釈

Azure Machine Learningを使用して、各機能が予測ラベルに与える影響の量を定量化する*説明者*を使用して、モデルを解釈できます。多くの一般的な説明者がいて、それぞれ異なる種類のモデリングアルゴリズムに適しています。しかし、それらを使用する基本的なアプローチは同じです。

## モデルの説明

Azure Machine Learningの外部でトレーニングされたモデルから始めましょう-下のセルを実行して、決定木分類モデルをトレーニングします。

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

トレーニングプロセスでは、ホールドバック検証データセットに基づいていくつかのモデル評価メトリクスが生成されたため、予測の正確性についてのアイデアが得られます。しかし、データの特徴は予測にどのように影響しますか？

### Azure ML Interpretability Libraryをインストールする
調べるには、まずAzure ML Interpretabilityライブラリをインストールします。これを使用して、Azure ML実験でトレーニングされていない場合や、Azure MLワークスペースに登録されていない場合でも、多くの一般的な種類のモデルを解釈できます。

In [ ]:
!pip install --upgrade azureml-interpret

### モデルの説明を取得する

ライブラリがインストールされていることに注意してください。モデルに適した説明者を取得しましょう。説明者には多くの種類があります。この例では、* Tabular Explainer *を使用します。これは、適切な[SHAP](https://github.com/slundberg/shap)モデルExplainerを呼び出すことで、多くの種類のモデルを説明するために使用できる"ブラックボックス" Explainerです。

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model, 
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### グローバル機能の重要性を取得

最初に行うことは、全体的な*機能の重要度*を評価することでモデルを説明することです。つまり、トレーニングデータセット全体に基づいて、各機能が予測に影響を与える程度を定量化します。

In [ ]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

機能の重要度がランク付けされ、最も重要な機能が最初にリストされます。

### Get Local Feature Importance

全体像はありますが、個々の観察結果の説明はどうですか？個々の予測の*ローカル*説明を生成し、各ラベルが可能なラベル値を予測する決定に影響する程度を定量化します。この場合、バイナリモデルであるため、2つのラベル（非糖尿病と糖尿病）があります。データセット内の個々の観測値について、これらのラベル値ごとに各特徴の影響を定量化できます。テストデータセットの最初の2つのケースを評価するだけです。

In [ ]:
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])



## Azure MLモデルのトレーニング実験に説明可能性を追加する

これまで見てきたように、Azure MLの外部でトレーニングされたモデルの説明を生成できます。ただし、実験を使用してAzure MLワークスペースでモデルをトレーニングする場合、モデルの説明を生成してログに記録できます。

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 実験を使用したモデルのトレーニングと説明

さて、実験を作成して、必要なファイルをローカルフォルダーに配置しましょう。この場合、糖尿病データの同じCSVファイルを使用してモデルをトレーニングします。

In [ ]:
import os, shutil
from azureml.core import Experiment

# Create a folder for the experiment files
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

次の機能を含むことを除いて、他のAzure MLトレーニングスクリプトに似たトレーニングスクリプトを作成します。

- 以前に使用したモデルの説明を生成するための同じライブラリがインポートされ、グローバルな説明の生成に使用されます
- **ExplanationClient**ライブラリーは、実験の出力に説明をアップロードするために使用されます

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# Complete the run
run.complete()

これで、推定器を使用してトレーニングスクリプトを実行し、実験を実行できます。 **azureml-interpret**ライブラリはトレーニング環境に含まれているため、スクリプトは** TabularExplainer **を作成でき、**azureml-contrib-interpret**パッケージが含まれているため、スクリプトは**ExplainerClient**クラス。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
env = Environment('diabetes-interpret-env')
env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (including the azureml-contrib-interpret package)
packages = CondaDependencies.create(conda_packages=['pip==19.3.1','scikit-learn','pandas'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-contrib-interpret'])

# Add the dependencies to the environment
env.python.conda_dependencies = packages

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              compute_target = 'local', # Use local compute
              environment_definition = env,
              entry_script='diabetes_training.py')

# Run the experiment
experiment = Experiment(workspace = ws, name = 'diabetes_train_and_explain')
run = experiment.submit(config=estimator)
RunDetails(run).show()
run.wait_for_completion()

## 機能の重要度の値を取得する

実験の実行が完了したら、**ExplanationClient**クラスを使用して、実行用に登録された説明から機能の重要度を取得できます。

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studioでモデルの説明を表示する

[実行の詳細]ウィジェットのリンクをクリックして、Azure Machine Learning Studioで実行を確認し、[*explainer*]タブを表示することもできます。次に：

1. explainerに**Tabular Explanation**を選択します。.
2. 全体的なグローバル機能の重要性を示す**Global Importance**チャートを表示します。
3. *swarm*、*violin*、または*box*プロットのテストデータからの各データポイントを示す**Summary Importance**チャートを表示します。
4. 個々のポイントを選択して、選択したデータポイントの個々の予測の**Local feature importance**を表示します。


**More Information**: For more information about using explainers in Azure ML, see [the documentation](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability). 